In [51]:
#reference: https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/
from pymongo import MongoClient
from nltk.tokenize import word_tokenize
import pandas as pd
import re
import operator 
import json
from collections import Counter
from nltk.corpus import stopwords
import string
from collections import defaultdict
import vincent

In [52]:
db_name= "test3"
collection = "trumpchina"
#TODO: automatically increment the list of stop words and hashtags
personalized_excluded_words = ['Trump', "Clinton", "MAGA", "TRUMP"]
personalized_excluded_hashtags = ['#Trump', '#trump', '#TRUMP']

In [53]:
client = MongoClient()
db = client[db_name]
coll= db[collection]

In [54]:
tweets = coll.find()
#db.collection.find({ age: { $exists: true } })
               
tweetsDf = pd.DataFrame(list(tweets))  


In [55]:
tweets= tweetsDf["text"]
tweets[1]

u'An informative and deeply arresting examination of how competition from China has impacted Americans. Fair trade? https://t.co/UdwsaClbvn'

In [56]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = "@gioazzi: example!  http://gioazzi.com #democrats #republicans "
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


['@gioazzi', ':', 'example', '!', 'http://gioazzi.com', '#democrats', '#republicans']


In [57]:
#TODO: escape emoticons errors

com = defaultdict(lambda : defaultdict(int))
count_all = Counter()
count_all_single = Counter()
count_all_hash = Counter()
count_all_terms = Counter()
punctuation = list(string.punctuation)

stop = stopwords.words('english') + punctuation + personalized_excluded_words +personalized_excluded_hashtags #+com

for index,row in tweetsDf.iterrows():

    terms_stop = [term for term in preprocess(row["text"]) if term not in stop]
    terms_all = [term for term in preprocess(row["text"]) if term not in stop]
    # Update the counter
    count_all.update(terms_all) 
    # Count terms only once, equivalent to Document Frequency
    terms_single = set(terms_all)
    count_all_single.update(terms_single)
    # Count hashtags only
    terms_hash = [term for term in preprocess(row["text"]) 
                  if term.startswith('#') and term not in stop]
    count_all_hash.update(terms_hash)
    # Count terms only (no hashtags, no mentions)
    terms_only = [term for term in preprocess(row["text"]) 
                  if term not in stop and
                  not term.startswith(('#', '@'))] 
                  # mind the ((double brackets))
                  # startswith() takes a tuple (not a list) if 
                  # we pass a list of inputs
    count_all_terms.update(terms_only)    
    # Build co-occurrence matrix
    for i in range(len(terms_only)-1):            
        for j in range(i+1, len(terms_only)):
            w1, w2 = sorted([terms_only[i], terms_only[j]])                
            if w1 != w2:
                com[w1][w2] += 1
                

                
com_max = []
# For each term, look for the most common co-occurrent terms
for t1 in com:
    t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:5]
    for t2, t2_count in t1_max_terms:
        com_max.append(((t1, t2), t2_count))
# Get the most frequent co-occurrences
terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)

print(terms_max[:5])                
                

[((u'\ud83c', u'\ud83d'), 3117), ((u'\ud83c', u'\udc4e'), 2180), ((u'\ud83d', u'\udc4e'), 2180), ((u'\ud83c', u'\udffd'), 2178), ((u'\ud83d', u'\udffd'), 2178)]


In [58]:
print(count_all.most_common(15))
print(count_all_single.most_common(15))
print(count_all_hash.most_common(15))
print(count_all_terms.most_common(15))

[(u'#China', 2148), (u'China', 1259), (u'\ud83c', 1057), (u'made', 432), (u'\u2019', 403), (u'amp', 333), (u'How', 314), (u'The', 283), (u'Donald', 279), (u'\ud83d', 270), (u'#Russia', 264), (u'#USA', 238), (u'via', 232), (u'#china', 229), (u'I', 173)]
[(u'#China', 2105), (u'China', 1150), (u'\u2019', 327), (u'How', 313), (u'made', 304), (u'amp', 284), (u'The', 275), (u'Donald', 275), (u'#Russia', 262), (u'#USA', 238), (u'via', 232), (u'#china', 229), (u'\ud83d', 175), (u"don't", 163), (u'Hillary', 160)]
[(u'#China', 2148), (u'#Russia', 264), (u'#USA', 238), (u'#china', 229), (u'#GOP', 150), (u'#LA', 141), (u'#NYC', 131), (u"#Trump's", 126), (u'#Bangladesh', 117), (u'#RT', 116), (u'#Haaretz', 116), (u'#Clinton', 113), (u'#foxnews', 110), (u'#NRA', 109), (u'#news', 109)]
[(u'China', 1259), (u'\ud83c', 1057), (u'made', 432), (u'\u2019', 403), (u'amp', 333), (u'How', 314), (u'The', 283), (u'Donald', 279), (u'\ud83d', 270), (u'via', 232), (u'I', 173), (u"don't", 165), (u'Hillary', 162), (u

In [59]:
'''
TO PLOT: 
Run the server:
python -m http.server 8888 # Python 3
python -m SimpleHTTPServer 8888 # Python 2

in chart.html uncomment this line: parse("term_freq.json");

open
http://localhost:8888/chart.html
'''

#prepare data to plot a barchar of the most frequent terms
word_freq = count_all_terms.most_common(20)
labels, freq = zip(*word_freq)
data = {'data': freq, 'x': labels}
bar = vincent.Bar(data, iter_idx='x')
bar.to_json('term_freq.json')

In [60]:
#prepare data to plot timeseries of some terms/hashtags in the dates_dict

'''
TO PLOT: 
Run the server:
python -m http.server 8888 # Python 3
python -m SimpleHTTPServer 8888 # Python 2

in chart.html uncomment this line: parse("time_chart.json");

open
http://localhost:8888/chart.html
'''
data_types= ['hashtag','words','accounts']
chosen_type = data_types[0]

time_span='10Min'
dates_dict = {'#trump2016':[], '#democrats':[] }

for index,row in tweetsDf.iterrows(): 
    if chosen_type== 'hashtag':
        terms = [term for term in preprocess(row['text']) if term.startswith('#')]
    if chosen_type== 'words':
        terms = [term for term in preprocess(row['text']) if not term.startswith(('#', '@'))]
    if chosen_type== 'accounts':
        terms = [term for term in preprocess(row['text']) if term.startswith('@')]

    # track when the hashtag is mentioned
    for key in dates_dict:
        if key in terms:
            dates_dict[key].append(row['raw']['created_at'])
             
# a list of "1" to count the hashtags
ones= {}
idx={}
dataframes={}
per_minute={}
for key in dates_dict:
    
    ones[key] = [1]*len(dates_dict[key])
    # the index of the series
    idx[key] = pd.DatetimeIndex(dates_dict[key])
    # the actual series (at series of 1s for the moment)
    dataframes[key] = pd.Series(ones[key], index=idx[key])
 
    # Resampling / bucketing
    per_minute[key] = dataframes[key].resample(time_span, how='sum').fillna(0)
print "done"
#print per_minute

done


In [61]:
# all the data together
match_data = per_minute
# we need a DataFrame, to accommodate multiple series
all_matches = pd.DataFrame(data=match_data,
                               index=per_minute['#trump2016'].index)
# Resampling 
all_matches = all_matches.resample(time_span, how='sum').fillna(0)
 
# and now the plotting
time_chart = vincent.Line(all_matches[list(match_data.keys())])
time_chart.axis_titles(x='Time', y='Freq')
time_chart.legend(title='Matches')
time_chart.to_json('time_chart.json')

In [62]:
#prepare data to plot geolocated graphs

#TO PLOT: http://geojson.io/#map=2/20.0/0.0

geo_data = {
    "type": "FeatureCollection",
    "features": []
}
for index,row in tweetsDf.iterrows():
    if row['raw']['coordinates']:
        geo_json_feature = {
            "type": "Feature",
            "geometry": row['raw']['coordinates'],
            "properties": {
                "text": row['text'],
                "created_at": row['raw']['created_at']
            }
        }
        geo_data['features'].append(geo_json_feature)
 
# Save geo data
with open('geo_data.json', 'w') as fout:
    fout.write(json.dumps(geo_data, indent=4))

In [63]:
'''
Bigrams: not used yet
from nltk import bigrams 
terms_bigram = bigrams(terms_stop)
'''

'\nBigrams: not used yet\nfrom nltk import bigrams \nterms_bigram = bigrams(terms_stop)\n'